In [1]:
import pandas as pd

In [14]:
df = pd.read_csv("./info_transportes.csv", sep=";", encoding="utf-8")


filtro = df[df["DATA_INICIO"] == "02-09-2016"]
print(filtro)


Empty DataFrame
Columns: [DATA_INICIO, DATA_FIM, CATEGORIA, LOCAL_INICIO, LOCAL_FIM, DISTANCIA, PROPOSITO]
Index: []


In [36]:

# Exemplo: lendo o CSV
df = pd.read_csv("./info_transportes.csv", sep=";", encoding="utf-8")

# Converter DATA_INICIO para datetime com o formato correto
df['DATA_INICIO'] = pd.to_datetime(df['DATA_INICIO'], format='%m-%d-%Y %H:%M')

# Agora para filtrar todas as linhas onde a DATA_INICIO tem data igual a 2 de setembro de 2016 (independente do horário)
filtro = df[df['DATA_INICIO'].dt.date == pd.to_datetime('2016-02-13').date()]

print(filtro)


            DATA_INICIO          DATA_FIM CATEGORIA      LOCAL_INICIO  \
99  2016-02-13 14:21:00  02-13-2016 14:41   Negocio              Cary   
100 2016-02-13 23:45:00   02-13-2016 0:01   Pessoal     East Elmhurst   
101 2016-02-13 00:50:00   02-13-2016 1:00   Pessoal   Jackson Heights   
102 2016-02-13 14:07:00  02-13-2016 14:40   Negocio     East Elmhurst   
103 2016-02-13 14:46:00  02-13-2016 15:03   Negocio           Midtown   
104 2016-02-13 16:35:00  02-13-2016 17:02   Negocio          New York   
105 2016-02-13 17:06:00  02-13-2016 17:29   Negocio  Long Island City   

            LOCAL_FIM  DISTANCIA PROPOSITO  
99        Morrisville         89   Reunião  
100   Jackson Heights         27       NaN  
101     East Elmhurst         18       NaN  
102          New York         81   Reunião  
103      Midtown West          2   Reunião  
104  Long Island City         13   Reunião  
105           Jamaica        139   Reunião  


In [35]:
spark = pd.read_csv("../dados/pyspark/part-00000-2cb1034b-85de-4cb9-97c2-500028bcf37b-c000.csv", sep=";", encoding="utf-8")
filtro = spark[spark["DT_REFE"] == "2016-02-13"]
print(filtro)

       DT_REFE  QT_CORR  QT_CORR_NEG  QT_CORR_PESS  VL_MAX_DIST  VL_MIN_DIST  \
29  2016-02-13        7            5             2          139            2   

    VL_AVG_DIST  QT_CORR_REUNI  QT_CORR_NAO_REUNI  
29        52.71              5                  2  


In [38]:
pandas= pd.read_csv("../dados/pandas/logbook.csv", sep=";", encoding="utf-8")
filtro = pandas[pandas["DT_REFE"] == "2016-02-13"]
print(filtro)


       DT_REFE  QT_CORR  QT_CORR_NEG  QT_CORR_PESS  VL_MAX_DIST  VL_MIN_DIST  \
29  2016-02-13        7            5             2          139            2   

    VL_AVG_DIST  QT_CORR_REUNI  QT_CORR_NAO_REUNI  
29        52.71              5                  2  


In [39]:
pandas_pq= pd.read_parquet("../dados/pandas/logbook.parquet")
filtro = pandas_pq[pandas_pq["DT_REFE"] == "2016-02-13"]
print(filtro)

       DT_REFE  QT_CORR  QT_CORR_NEG  QT_CORR_PESS  VL_MAX_DIST  VL_MIN_DIST  \
29  2016-02-13        7            5             2          139            2   

    VL_AVG_DIST  QT_CORR_REUNI  QT_CORR_NAO_REUNI  
29        52.71              5                  2  


In [50]:
spark_pq = pd.read_parquet("../dados/pyspark/logbook.parquet")
spark_pq["DT_REFE"] = pd.to_datetime(spark_pq["DT_REFE"])
#spark_pq.head(90)
filtro = spark_pq[spark_pq["DT_REFE"] == "2016-02-13"]
print(filtro)

      DT_REFE  QT_CORR  QT_CORR_NEG  QT_CORR_PESS  VL_MAX_DIST  VL_MIN_DIST  \
29 2016-02-13        7            5             2          139            2   

   VL_AVG_DIST  QT_CORR_REUNI  QT_CORR_NAO_REUNI  
29       52.71              5                  2  


In [ ]:
df['DATA_INICIO'] = pd.to_datetime(df['DATA_INICIO'])

#garantindo a inexistência de nulos
df['DATA_INICIO'].isna().sum()

np.int64(0)

In [75]:
df['DT_REFE'] = df['DATA_INICIO'].dt.strftime('%Y-%m-%d')


new_df = df.groupby('DT_REFE').agg(
    QT_CORR_NEG=('CATEGORIA', lambda x:(x == 'Negocio').sum()),
    QT_CORR_PESS=('CATEGORIA', lambda x:(x == 'Pessoal').sum()),
    VL_MAX_DIST=('DISTANCIA', 'max'),
    VL_MIN_DIST=('DISTANCIA', 'min'),
    VL_AVG_DIST=('DISTANCIA', 'mean'),
    QT_CORR_REUNI=('PROPOSITO', lambda x:(x == 'Reunião').sum()),
    QT_CORR_NAO_REUNI=('PROPOSITO', lambda x: (x != 'Reunião').sum())
                         
).reset_index()

new_df['QT_CORR'] = df.groupby('DT_REFE').size().values

new_df = new_df[[
    'DT_REFE', 'QT_CORR', 'QT_CORR_NEG', 'QT_CORR_PESS',
    'VL_MAX_DIST', 'VL_MIN_DIST', 'VL_AVG_DIST',
    'QT_CORR_REUNI', 'QT_CORR_NAO_REUNI'
]]

new_df['VL_AVG_DIST'] = new_df['VL_AVG_DIST'].round(2)

new_df.head()


,DT_REFE,QT_CORR,QT_CORR_NEG,QT_CORR_PESS,VL_MAX_DIST,VL_MIN_DIST,VL_AVG_DIST,QT_CORR_REUNI,QT_CORR_NAO_REUNI
0,2016-01-01,1,1,0,51,51,51.00,0,1
1,2016-01-02,2,2,0,48,5,26.50,0,2
2,2016-01-05,1,1,0,47,47,47.00,1,0
3,2016-01-06,3,3,0,637,43,250.33,1,2
4,2016-01-07,1,1,0,8,8,8.00,1,0


In [ ]:
new_df.to_csv("./info_corridas_do_dia.csv", sep=";", header=True)

new_df.to_parquet("./info_corridas_do_dia.parquet")

In [26]:
pandas = pd.read_parquet("D:\\Ana Paula\\f1rst-challenge\\dados\\pandas\\logbook.parquet")
pandas_sorted = pandas.sort_values(by='DT_REFE', ascending=True)
pandas_sorted.head()



,DT_REFE,QT_CORR,QT_CORR_NEG,QT_CORR_PESS,VL_MAX_DIST,VL_MIN_DIST,VL_AVG_DIST,QT_CORR_REUNI,QT_CORR_NAO_REUNI
0,2016-01-01,1,1,0,51,51,51.00,0,1
1,2016-01-02,2,2,0,48,5,26.50,0,2
2,2016-01-05,1,1,0,47,47,47.00,1,0
3,2016-01-06,3,3,0,637,43,250.33,1,2
4,2016-01-07,1,1,0,8,8,8.00,1,0


In [25]:
spark = pd.read_parquet("D:\\Ana Paula\\f1rst-challenge\\dados\\parquet\\part-00000-3327f586-100a-432f-91ed-da6eee7006a3-c000.snappy.parquet")
df_spark_sorted = spark.sort_values(by='DT_REFE', ascending=True)
df_spark_sorted.head()

,DT_REFE,QT_CORR,QT_CORR_NEG,QT_CORR_PESS,VL_MAX_DIST,VL_MIN_DIST,VL_AVG_DIST,QT_CORR_REUNI,QT_CORR_NAO_REUNI
14,2016-01-01,1,1,0,51,51,51.000000,0,1
212,2016-01-02,2,2,0,48,5,26.500000,0,1
69,2016-01-05,1,1,0,47,47,47.000000,1,0
70,2016-01-06,3,3,0,637,43,250.333333,1,2
262,2016-01-07,1,1,0,8,8,8.000000,1,0


In [13]:
spark.head()

,DT_REFE,QT_CORR,QT_CORR_NEG,QT_CORR_PESS,VL_MAX_DIST,VL_MIN_DIST,VL_AVG_DIST,QT_CORR_REUNI,QT_CORR_NAO_REUNI
0,None,0,1076,77,3103,1,91.828274,187,464
